# Traffic Violation
Create a model using logitic regression to determine the property damage based on various factors

Hypothesis testing 

Create a model using random forest to evaluate the description and determine if there are any key fators that impact property damage 

Evaluate the date and time if the they play any significant role.

How does location play a role, are some locations more prone than others 

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

df_raw = pd.read_csv('/Users/yogini/Downloads/Traffic_Violations_train.csv')
df = df_raw.dropna() 
print df.head()
df.columns

   Unnamed: 0 Date Of Stop Time Of Stop Agency  \
0      231432   10/06/2013     22:58:00    MCP   
1      375258   09/13/2015     01:41:00    MCP   
2      902304   09/03/2014     23:54:00    MCP   
3      955895   07/05/2015     12:45:00    MCP   
4      701179   05/11/2015     21:47:00    MCP   

                             SubAgency  \
0          3rd district, Silver Spring   
1  Headquarters and Special Operations   
2                4th district, Wheaton   
3             5th district, Germantown   
4                4th district, Wheaton   

                                         Description  \
0  DRIVE ACROSS PRIVATE PROPERTY FOR THE PURPOSE ...   
1  NEGLIGENT DRIVING VEHICLE IN CARELESS AND IMPR...   
2  FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...   
3  DRIVING VEHICLE ON HIGHWAY WITHOUT CURRENT REG...   
4  RECKLESS DRIVING VEHICLE IN WANTON AND WILLFUL...   

                                   Location   Latitude  Longitude  Accident  \
0           BRIGGS CHANEY RD A

Index([u'Unnamed: 0', u'Date Of Stop', u'Time Of Stop', u'Agency',
       u'SubAgency', u'Description', u'Location', u'Latitude', u'Longitude',
       u'Accident', u'Belts', u'Personal Injury', u'Property Damage', u'Fatal',
       u'Commercial License', u'HAZMAT', u'Commercial Vehicle', u'Alcohol',
       u'Work Zone', u'State', u'VehicleType', u'Year', u'Make', u'Model',
       u'Color', u'Violation Type', u'Charge', u'Article',
       u'Contributed To Accident', u'Race', u'Gender', u'Driver City',
       u'Driver State', u'DL State', u'Arrest Type', u'Geolocation',
       u'stop-year', u'stop-month'],
      dtype='object')

In [10]:
df.head()

,Unnamed: 0,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation,stop-year,stop-month
0,231432,10/06/2013,22:58:00,MCP,"3rd district, Silver Spring",DRIVE ACROSS PRIVATE PROPERTY FOR THE PURPOSE ...,BRIGGS CHANEY RD AT CASTLE BLVD,39.082817,-76.947782,0,...,0,WHITE,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.0828166666667, -76.9477816666667)",2013,10
1,375258,09/13/2015,01:41:00,MCP,Headquarters and Special Operations,NEGLIGENT DRIVING VEHICLE IN CARELESS AND IMPR...,9880 ROCKVILLE PIK / 5100 KING CHARLES W,38.983630,-77.093272,0,...,0,WHITE,M,PIKESVILLE,MD,MD,A - Marked Patrol,"(38.98363, -77.0932716666667)",2015,9
2,902304,09/03/2014,23:54:00,MCP,"4th district, Wheaton",FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...,GEORGIA AVE / CONNECTICUT,39.087468,-77.079330,0,...,0,BLACK,M,BALTIMORE,MD,MD,A - Marked Patrol,"(39.0874683333333, -77.07933)",2014,9
3,955895,07/05/2015,12:45:00,MCP,"5th district, Germantown",DRIVING VEHICLE ON HIGHWAY WITHOUT CURRENT REG...,GERMANTOWN ROAD @ CRYSTAL ROCK DRIVE,39.184212,-77.255140,0,...,0,WHITE,F,ROCKVILLE,MD,MD,A - Marked Patrol,"(39.1842116666667, -77.25514)",2015,7
4,701179,05/11/2015,21:47:00,MCP,"4th district, Wheaton",RECKLESS DRIVING VEHICLE IN WANTON AND WILLFUL...,N/B I-270 PRIOR TO MIDDLEBROOK RD,39.165833,-77.240402,0,...,0,BLACK,M,GERMANTOWN,MD,MD,B - Unmarked Patrol,"(39.1658333333333, -77.2404016666667)",2015,5


Calculate the Odds ratio of Property damage where the driver ins drunk. 

In [12]:
pd.crosstab(df['Property Damage'],df['Alcohol'])

Alcohol,0,1
Property Damage,,
0,534044,1045
1,9437,68


In [14]:
od_p1=68.0/1045
od_p2=9437.0/534044
od_ratio= od_p1/od_p2

In [15]:
od_ratio

3.682440237018799

# The Odds of property damage are 3.6 times higher if the driver was drunk driving.

In [53]:
df['intercept'] = 1.0

In [67]:
df.columns

Index([u'Unnamed: 0', u'Date Of Stop', u'Time Of Stop', u'Agency',
       u'SubAgency', u'Description', u'Location', u'Latitude', u'Longitude',
       u'Accident', u'Belts', u'Personal Injury', u'Property Damage', u'Fatal',
       u'Commercial License', u'HAZMAT', u'Commercial Vehicle', u'Alcohol',
       u'Work Zone', u'State', u'VehicleType', u'Year', u'Make', u'Model',
       u'Color', u'Violation Type', u'Charge', u'Article',
       u'Contributed To Accident', u'Race', u'Gender', u'Driver City',
       u'Driver State', u'DL State', u'Arrest Type', u'Geolocation',
       u'stop-year', u'stop-month', u'intercept'],
      dtype='object')

In [84]:
#df.columns[17]
#train_cols=df.columns[17]
train_cols1=df.loc[:,['Alcohol','Belts','HAZMAT']]
train_cols=train_cols1.columns[:]
#train_cols=df.columns[:"Alcohol":]
#train_cols
#train_cols.append(df.columns[14:19])

In [85]:
train_cols

Index([u'Alcohol', u'Belts', u'HAZMAT'], dtype='object')

In [86]:
logit = sm.Logit(df['Property Damage'], df[train_cols])

In [87]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.674680
         Iterations 7


In [82]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:        Property Damage   No. Observations:               544594
Model:                          Logit   Df Residuals:                   544593
Method:                           MLE   Df Model:                            0
Date:                Mon, 16 Jan 2017   Pseudo R-squ.:                  -6.870
Time:                        15:12:16   Log-Likelihood:            -3.7697e+05
converged:                       True   LL-Null:                       -47900.
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Alcohol       -2.7323      0.125    -21.832      0.000        -2.978    -2.487


In [88]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:        Property Damage   No. Observations:               544594
Model:                          Logit   Df Residuals:                   544591
Method:                           MLE   Df Model:                            2
Date:                Mon, 16 Jan 2017   Pseudo R-squ.:                  -6.671
Time:                        15:13:11   Log-Likelihood:            -3.6743e+05
converged:                       True   LL-Null:                       -47900.
                                        LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Alcohol       -2.7233      0.125    -21.755      0.000        -2.969    -2.478
Belts         -2.7425      0.029    -93.531      0.000        -2.800    -2.685
HAZMAT        -3.2478      0.721     -4.507      0.0